In [ ]:
import pandas as pd
import numpy as np
import glob
import cv2
import matplotlib.pyplot as plt
import os
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import pickle

In [ ]:
png_files = []
folder_path = r"/home/legendarygene/Desktop/assn2/train/"
for i in range(0,2000):
    file = glob.glob(f"{folder_path}{i}.png")
    png_files.append(file)

In [ ]:
cropped_train = []

def cropper(i):
    image = Image.open(png_files[i][0])
    width, height = image.size

    crop_region = (width - 140, 10, width - 50, height)
    ci = image.crop(crop_region)
    ci = np.asarray(ci)
    
    _,img = cv2.threshold(ci,165,255,cv2.THRESH_BINARY)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _,img = cv2.threshold(img,254,255,cv2.THRESH_BINARY)
    kernel = np.ones((5,5), np.uint8)
    img_erode = cv2.dilate(img, kernel)
    cropped_train.append(img_erode)

In [ ]:
for i in range(2000):
    cropper(i)
x_train = np.array(cropped_train)
file = open(r"/home/legendarygene/Desktop/assn2/train/labels.txt", "r")
temp = file.read().splitlines()
y_train = np.array(temp)

In [ ]:
Y_train =[]
for i in range(2000):
    if y_train[i] == 'ODD':
        Y_train.append(1)
    else:
        Y_train.append(0)
Y_train = np.array(Y_train)

In [ ]:
X_trn, x_tst,Y_trn, y_tst = train_test_split(x_train,Y_train,test_size=0.25)

In [ ]:
X_trn = X_trn / 255.0

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(2, kernel_size=(3, 3), activation='relu', input_shape=(90, 90, 1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(4, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(8, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(8, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(X_trn, Y_trn, epochs=30, batch_size=32)
# model.save_weights('model.h5')
weights_file = "final.pkl"
with open(weights_file, "wb") as f:
    pickle.dump(model, f)


In [ ]:
predictions = model.predict(x_tst)
count = 0
for i in range(500):
    if predictions[i] == y_tst[i]:
        count = count + 1
count/5.0